In [45]:
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

## Загружаем данные из файла
## Данные за 01.03.22 - 15.05.22

In [46]:
data=pd.read_csv('16.05.22 full_version_evname_json_date.csv')
pd.set_option('display.max_rows', 10) # None
pd.set_option('display.max_columns', 50) # None
data.head()

,app_version_name,event_name,event_json,event_datetime
0,1.4.1,app_start,"{""balance"":""450"",""days_in_game"":""0"",""PuzzleCom...",2022-04-21 12:58:04
1,1.4.1,balance_change,"{""Currency"":""coins"",""Balance"":""450"",""balance"":...",2022-04-21 12:58:03
2,1.4.1,game_puzzle_click,"{""category_id"":""13"",""puzzle_id"":""38"",""Order_nu...",2022-04-21 12:58:45
3,1.4.1,game_puzzle_pause,"{""puzzle_id"":""38"",""Order_number"":""38"",""categor...",2022-04-21 12:59:03
4,1.4.1,game_puzzle_receive,"{""place"":""daily"",""category_id"":""13"",""Order_num...",2022-04-21 12:58:47


## Оставляем в датафрейме только event_name связанные с пазлами:  
'show_image'  
'game_puzzle_start'  
'game_puzzle_complete'  
'game_puzzle_quit'  
'game_puzzle_concat'  
'game_puzzle_receive'  
'game_puzzle_click'  
'game_puzzle_start_continue'  
'game_puzze_onscreen'  
## В jsone осталяем нужное и делаем один датафрейм

In [47]:
data=data[data['event_name'].isin(\
    ['show_image', 'game_puzzle_start', 'game_puzzle_complete', 'game_puzzle_quit', 'game_puzzle_concat', 'game_puzzle_receive', 'game_puzzle_click', 'game_puzzle_start_continue', 'game_puzze_onscreen']\
        )].reset_index(drop=True)

# развернем json
data_json = pd.json_normalize(list(map(json.loads, data.event_json)))

# сформируем фрейм с которым будем работать
data_work = data[['event_name', 'event_datetime', 'app_version_name']]

# добавим столбец с единым идентификатором пазла
data_work['cat_puzzle_id'] = data_json.category_id.astype(str) + '_' + data_json.puzzle_id.astype(str)

# сконкатим фреймы data и json
data_work = pd.concat([data_work, data_json[[\
    'category_id','place','days_in_game','PuzzleComplete','PuzzleStart','Rotate','Difficulty','completed','background','Progress','Continue','started','available','Percent','Concat','Placed']\
        ]], axis=1)

# Преобразуем типы данных
data_work['PuzzleComplete'] =pd.to_numeric(data_work.PuzzleComplete)
data_work['PuzzleStart'] =pd.to_numeric(data_work.PuzzleStart)

## Датафрейм, со всем необходимым

In [48]:
data_work.head(3)

,event_name,event_datetime,app_version_name,cat_puzzle_id,category_id,place,days_in_game,PuzzleComplete,PuzzleStart,Rotate,Difficulty,completed,background,Progress,Continue,started,available,Percent,Concat,Placed
0,game_puzzle_click,2022-04-21 12:58:45,1.4.1,13_38,13,category,0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game_puzzle_receive,2022-04-21 12:58:47,1.4.1,13_38,13,daily,0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,show_image,2022-04-21 09:34:57,1.4.1,1_100,1,category,0,0.0,1.0,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


## Любопытно посомтреть как изменятся значения конверсий если оставить place = daily or category

In [81]:
#data_work = data_work[data_work['place'].isin(['daily', 'catalog'])]
data_work['place'].value_counts()

daily            2916492
category         1679385
in_progress       298355
inprogress        197248
free_category     120504
completed          93978
none               19117
Name: place, dtype: int64

## Опишем функцию которая посчитает для переданого датафрейма:
Показы  
Старты  
Комплиты  
Конверсию показов в старты  
Конверсию стартов в комплиты  
Конверсию стартов в quitы  
__Все значения считаются для событий из daily, catalog и все остальные, для не собранных ранее пазлов (completed !="1") , заданного набора Difficulty, по заданным версиям__

In [50]:
def conversion_count_simply(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)
    
    return data_count_all



## Опишем функцию которая посчитает для переданого датафрейма ПЕРВЫЕ конверсии:


In [51]:
def conversion_count_first(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1" and PuzzleStart == 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start" and PuzzleStart == 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete" and PuzzleComplete == 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit" and PuzzleStart == 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)
    
    return data_count_all

## Опишем функцию которая посчитает для переданого датафрейма ПЕРВЫЕ ПЛЮС конверсии:

In [52]:
def conversion_count_first_plus(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1" and PuzzleStart >0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start" and PuzzleStart > 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete" and PuzzleComplete > 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit" and PuzzleStart > 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)
    
    return data_count_all

# Расчитаем конверсии

## Итоговая таблица конверсий для всех стартов, по версиям, для всех Difficulty, по пазлам

In [53]:
conversion_table_simply = pd.concat([conversion_count_simply(data_work, 'full'),
        conversion_count_simply(data_work, '1.3.33', version=['1.3.33']).iloc[:,2:10],
            conversion_count_simply(data_work, '1.4', version=['1.4']).iloc[:,2:10],
                conversion_count_simply(data_work, '1.4.1', version=['1.4.1']).iloc[:,2:10],
                    conversion_count_simply(data_work, '1.4.3', version=['1.4.3']).iloc[:,2:10]
                        ], axis=1)

conversion_table_simply.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_complete_full,conv_to_quit_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_complete_1.3.33,conv_to_quit_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_complete_1.4,conv_to_quit_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_complete_1.4.1,conv_to_quit_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_complete_1.4.3,conv_to_quit_1.4.3
371,1_33,1,66128,3363,2601,497.0,5.09,77.34,14.78,519,14.0,11.0,2.0,2.70,78.57,14.29,31613,1540,1152,191.0,4.87,74.81,12.40,21858,1250,1002,202.0,5.72,80.16,16.16,12138,559,436.0,102.0,4.61,78.00,18.25
873,31_8,31,55361,2804,2091,457.0,5.06,74.57,16.30,487,25.0,14.0,3.0,5.13,56.00,12.00,32632,1522,1084,192.0,4.66,71.22,12.61,21782,1253,982,260.0,5.75,78.37,20.75,460,4,11.0,2.0,0.87,275.00,50.00
964,34_1,34,54518,2592,1919,490.0,4.75,74.04,18.90,442,14.0,10.0,1.0,3.17,71.43,7.14,31409,1368,961,223.0,4.36,70.25,16.30,22151,1196,926,260.0,5.40,77.42,21.74,516,14,22.0,6.0,2.71,157.14,42.86
496,21_3,21,50018,3188,2229,590.0,6.37,69.92,18.51,475,23.0,14.0,0.0,4.84,60.87,0.00,28837,1762,1176,280.0,6.11,66.74,15.89,20332,1398,1031,308.0,6.88,73.75,22.03,374,5,8.0,2.0,1.34,160.00,40.00
815,31_27,31,15445,589,491,74.0,3.81,83.36,12.56,43,2.0,2.0,0.0,4.65,100.00,0.00,1039,64,56,5.0,6.16,87.50,7.81,1116,70,67,7.0,6.27,95.71,10.00,13247,453,366.0,62.0,3.42,80.79,13.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,7_71,7,543,34,29,4.0,6.26,85.29,11.76,40,4.0,3.0,0.0,10.00,75.00,0.00,264,16,14,1.0,6.06,87.50,6.25,159,11,9,3.0,6.92,81.82,27.27,80,3,3.0,0.0,3.75,100.00,0.00
1289,7_68,7,542,29,25,3.0,5.35,86.21,10.34,39,1.0,1.0,0.0,2.56,100.00,0.00,259,13,10,1.0,5.02,76.92,7.69,180,13,12,2.0,7.22,92.31,15.38,64,2,2.0,0.0,3.12,100.00,0.00
1277,7_57,7,529,26,23,2.0,4.91,88.46,7.69,32,1.0,1.0,0.0,3.12,100.00,0.00,247,13,10,1.0,5.26,76.92,7.69,192,7,7,1.0,3.65,100.00,14.29,58,5,5.0,0.0,8.62,100.00,0.00
555,21_83,21,525,45,35,2.0,8.57,77.78,4.44,23,1.0,1.0,0.0,4.35,100.00,0.00,214,21,17,0.0,9.81,80.95,0.00,194,19,13,2.0,9.79,68.42,10.53,94,4,4.0,0.0,4.26,100.00,0.00


## Итоговая таблица конверсий для ПЕРВЫХ стартов, по версиям, для всех Difficulty = 36, по пазлам

In [54]:
diff = ['36']
conversion_table_first = pd.concat([conversion_count_first(data_work, 'full', difficulty=diff),
        conversion_count_first(data_work, '1.3.33', version=['1.3.33'], difficulty=diff).iloc[:,2:10],
            conversion_count_first(data_work, '1.4', version=['1.4'], difficulty=diff).iloc[:,2:10],
                conversion_count_first(data_work, '1.4.1', version=['1.4.1'], difficulty=diff).iloc[:,2:10],
                    conversion_count_first(data_work, '1.4.3', version=['1.4.3'], difficulty=diff).iloc[:,2:10]
                        ], axis=1)
conversion_table_first.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_complete_full,conv_to_quit_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_complete_1.3.33,conv_to_quit_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_complete_1.4,conv_to_quit_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_complete_1.4.1,conv_to_quit_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_complete_1.4.3,conv_to_quit_1.4.3
371,1_33,1,15374,445.0,293.0,65.0,2.89,65.84,14.61,5.0,0.0,0.0,0.0,0.0,0.0,0.0,6329,152.0,100.0,16.0,2.40,65.79,10.53,5998,176.0,114.0,34.0,2.93,64.77,19.32,2881,112.0,76.0,13.0,3.89,67.86,11.61
873,31_8,31,12970,582.0,419.0,83.0,4.49,71.99,14.26,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6515,283.0,205.0,27.0,4.34,72.44,9.54,6277,295.0,212.0,55.0,4.70,71.86,18.64,12,0.0,0.0,0.0,0.00,0.00,0.00
964,34_1,34,12680,522.0,346.0,88.0,4.12,66.28,16.86,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6387,254.0,169.0,30.0,3.98,66.54,11.81,6101,266.0,176.0,58.0,4.36,66.17,21.80,30,0.0,0.0,0.0,0.00,0.00,0.00
496,21_3,21,12096,902.0,560.0,146.0,7.46,62.08,16.19,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6143,482.0,299.0,75.0,7.85,62.03,15.56,5784,411.0,253.0,71.0,7.11,61.56,17.27,10,0.0,1.0,0.0,0.00,0.00,0.00
405,1_64,1,3259,969.0,708.0,142.0,29.73,73.07,14.65,6.0,0.0,0.0,0.0,0.0,0.0,0.0,192,5.0,5.0,1.0,2.60,100.00,20.00,131,2.0,1.0,0.0,1.53,50.00,0.00,2923,962.0,702.0,141.0,32.91,72.97,14.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,7_37,7,48,2.0,0.0,0.0,4.17,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,0.0,0.0,0.0,0.00,0.00,0.00,19,2.0,0.0,0.0,10.53,0.00,0.00,5,0.0,0.0,0.0,0.00,0.00,0.00
1233,7_17,7,46,1.0,1.0,1.0,2.17,100.00,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,1.0,1.0,0.0,4.76,100.00,0.00,17,0.0,0.0,1.0,0.00,0.00,0.00,6,0.0,0.0,0.0,0.00,0.00,0.00
1292,7_70,7,46,3.0,2.0,0.0,6.52,66.67,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,2.0,2.0,0.0,10.53,100.00,0.00,21,1.0,0.0,0.0,4.76,0.00,0.00,3,0.0,0.0,0.0,0.00,0.00,0.00
1290,7_69,7,45,2.0,2.0,0.0,4.44,100.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,2.0,1.0,0.0,10.00,50.00,0.00,19,0.0,0.0,0.0,0.00,0.00,0.00,4,0.0,1.0,0.0,0.00,0.00,0.00


## Итоговая таблица конверсий для первых ПЛЮС стартов, по версиям, для всех Difficulty = 36, по пазлам

In [55]:
diff = ['36']
conversion_table_first_plus = pd.concat([conversion_count_first_plus(data_work, 'full', difficulty=diff),
        conversion_count_first_plus(data_work, '1.3.33', version=['1.3.33'], difficulty=diff).iloc[:,2:10],
            conversion_count_first_plus(data_work, '1.4', version=['1.4'], difficulty=diff).iloc[:,2:10],
                conversion_count_first_plus(data_work, '1.4.1', version=['1.4.1'], difficulty=diff).iloc[:,2:10],
                    conversion_count_first_plus(data_work, '1.4.3', version=['1.4.3'], difficulty=diff).iloc[:,2:10]
                        ], axis=1)
conversion_table_first_plus.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_complete_full,conv_to_quit_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_complete_1.3.33,conv_to_quit_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_complete_1.4,conv_to_quit_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_complete_1.4.1,conv_to_quit_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_complete_1.4.3,conv_to_quit_1.4.3
371,1_33,1,50754,2100,1816,204.0,4.14,86.48,9.71,353,7.0,7.0,0.0,1.98,100.00,0.0,25284,1008,839,78.0,3.99,83.23,7.74,15860,772,698,85.0,4.87,90.41,11.01,9257,313.0,272.0,41.0,3.38,86.90,13.10
873,31_8,31,42391,1466,1260,167.0,3.46,85.95,11.39,321,11.0,8.0,0.0,3.43,72.73,0.0,26117,808,656,80.0,3.09,81.19,9.90,15505,647,590,87.0,4.17,91.19,13.45,448,0.0,6.0,0.0,0.00,0.00,0.00
964,34_1,34,41838,1295,1134,168.0,3.10,87.57,12.97,280,6.0,5.0,0.0,2.14,83.33,0.0,25022,682,567,78.0,2.73,83.14,11.44,16050,601,551,89.0,3.74,91.68,14.81,486,6.0,11.0,1.0,1.23,183.33,16.67
496,21_3,21,37922,1519,1255,202.0,4.01,82.62,13.30,316,6.0,4.0,0.0,1.90,66.67,0.0,22694,839,659,92.0,3.70,78.55,10.97,14548,671,588,110.0,4.61,87.63,16.39,364,3.0,4.0,0.0,0.82,133.33,0.00
815,31_27,31,12217,315,273,24.0,2.58,86.67,7.62,41,0.0,0.0,0.0,0.00,0.00,0.0,935,26,23,0.0,2.78,88.46,0.00,1014,34,33,0.0,3.35,97.06,0.00,10227,255.0,217.0,24.0,2.49,85.10,9.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,7_68,7,477,11,10,2.0,2.31,90.91,18.18,35,0.0,0.0,0.0,0.00,0.00,0.0,228,5,4,1.0,2.19,80.00,20.00,159,5,5,1.0,3.14,100.00,20.00,55,1.0,1.0,0.0,1.82,100.00,0.00
514,21_46,21,473,14,13,1.0,2.96,92.86,7.14,22,1.0,1.0,0.0,4.55,100.00,0.0,226,5,5,0.0,2.21,100.00,0.00,134,6,5,1.0,4.48,83.33,16.67,91,2.0,2.0,0.0,2.20,100.00,0.00
1277,7_57,7,470,12,10,2.0,2.55,83.33,16.67,28,0.0,0.0,0.0,0.00,0.00,0.0,225,5,3,1.0,2.22,60.00,20.00,168,4,4,1.0,2.38,100.00,25.00,49,3.0,3.0,0.0,6.12,100.00,0.00
1290,7_69,7,464,12,15,1.0,2.59,125.00,8.33,23,0.0,0.0,0.0,0.00,0.00,0.0,233,6,8,1.0,2.58,133.33,16.67,169,4,5,0.0,2.37,125.00,0.00,39,2.0,2.0,0.0,5.13,100.00,0.00


## Выведем средние по всем конверсиям и версиям в одну таблицу.  
### (starts hits, % - процент наблюдений стартов приходящихся на версию приложения)

In [56]:
conv_list = ['conv_to_start_full','conv_to_complete_full','conv_to_quit_full',\
    'conv_to_start_1.3.33','conv_to_complete_1.3.33','conv_to_quit_1.3.33',\
        'conv_to_start_1.4','conv_to_complete_1.4','conv_to_quit_1.4',\
           'conv_to_start_1.4.1','conv_to_complete_1.4.1','conv_to_quit_1.4.1',\
               'conv_to_start_1.4.3','conv_to_complete_1.4.3','conv_to_quit_1.4.3']

In [57]:
conversion_only = conversion_table_simply[conv_list]

conv_matrix_simply = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start', 'conv_to_complete', 'conv_to_quit'])
counter = 0
for i in range(5):
    x= []
    for j in range(3):
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        counter+=1
    conv_matrix_simply.iloc[:,i] = x
conv_matrix_simply

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start,4.71,3.71,4.56,5.32,4.26
conv_to_complete,85.34,63.48,79.65,89.88,94.63
conv_to_quit,12.45,1.84,11.73,13.86,11.82


In [58]:
conversion_only = conversion_table_first[conv_list]

conv_matrix_first = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start_first', 'conv_to_complete_first', 'conv_to_quit_first'])
counter = 0
for i in range(5):
    x= []
    for j in range(3):
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        counter+=1
    conv_matrix_first.iloc[:,i] = x
conv_matrix_first

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start_first,3.59,1.47,4.23,4.14,0.81
conv_to_complete_first,68.54,3.18,60.56,56.05,10.08
conv_to_quit_first,13.09,0.32,10.59,11.72,2.17


In [59]:
conversion_only = conversion_table_first_plus[conv_list]

conv_matrix_first_plus = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start_first_plus', 'conv_to_complete_first_plus', 'conv_to_quit_first_plus'])
counter = 0
for i in range(5):
    x= []
    for j in range(3):
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        counter+=1
    conv_matrix_first_plus.iloc[:,i] = x
conv_matrix_first_plus

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start_first_plus,2.69,1.55,2.44,3.31,2.55
conv_to_complete_first_plus,92.20,32.93,87.19,97.37,92.19
conv_to_quit_first_plus,7.44,0.36,7.13,8.33,6.37


In [60]:
result_matrix = pd.concat([conv_matrix_simply, conv_matrix_first, conv_matrix_first_plus])
start_hits_list = ['starts_full', 'starts_1.3.33' ,'starts_1.4' ,'starts_1.4.1','starts_1.4.3']

start_hits=[]
for i in start_hits_list:
    start_hits.append(round(conversion_table_simply[i].sum()*100/conversion_table_simply[start_hits_list[0]].sum(),1))
result_matrix.loc['starts hits, %'] = start_hits
result_matrix

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start,4.71,3.71,4.56,5.32,4.26
conv_to_complete,85.34,63.48,79.65,89.88,94.63
conv_to_quit,12.45,1.84,11.73,13.86,11.82
conv_to_start_first,3.59,1.47,4.23,4.14,0.81
conv_to_complete_first,68.54,3.18,60.56,56.05,10.08
conv_to_quit_first,13.09,0.32,10.59,11.72,2.17
conv_to_start_first_plus,2.69,1.55,2.44,3.31,2.55
conv_to_complete_first_plus,92.20,32.93,87.19,97.37,92.19
conv_to_quit_first_plus,7.44,0.36,7.13,8.33,6.37
"starts hits, %",100.00,1.70,42.50,38.00,17.70


## Таблица конверсий разных сложностей (без разбивки по версиям)

In [61]:
dif_ty=['36','49','144','256','441']

conversion_table_difficulty = pd.concat([conversion_count_simply(data_work, 'full', dif_ty),\
    conversion_count_simply(data_work, '36', ['36']).iloc[:,2:10],\
        conversion_count_simply(data_work, '49', ['49']).iloc[:,2:10],\
            conversion_count_simply(data_work, '144', ['144']).iloc[:,2:10],\
                conversion_count_simply(data_work, '256', ['256']).iloc[:,2:10],\
                    conversion_count_simply(data_work, '441', ['441']).iloc[:,2:10]], axis=1)

conversion_table_difficulty.sort_values('shows_full', ascending=False).head()

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_complete_full,conv_to_quit_full,shows_36,starts_36,complets_36,quits_36,conv_to_start_36,conv_to_complete_36,conv_to_quit_36,shows_49,starts_49,complets_49,quits_49,conv_to_start_49,conv_to_complete_49,conv_to_quit_49,shows_144,starts_144,complets_144,quits_144,conv_to_start_144,conv_to_complete_144,conv_to_quit_144,shows_256,starts_256,complets_256,quits_256,conv_to_start_256,conv_to_complete_256,conv_to_quit_256,shows_441,starts_441,complets_441,quits_441,conv_to_start_441,conv_to_complete_441,conv_to_quit_441
371,1_33,1,66128,3363,2601,497.0,5.09,77.34,14.78,66128,2545,2109,269.0,3.85,82.87,10.57,66128,533,426,87.0,0.81,79.92,16.32,66128,184.0,57.0,76.0,0.28,30.98,41.30,66128,37.0,7.0,17.0,0.06,18.92,45.95,66128,64.0,2.0,48.0,0.10,3.12,75.00
873,31_8,31,55361,2804,2091,457.0,5.06,74.57,16.30,55361,2048,1679,250.0,3.70,81.98,12.21,55361,433,337,78.0,0.78,77.83,18.01,55361,172.0,64.0,48.0,0.31,37.21,27.91,55361,71.0,8.0,47.0,0.13,11.27,66.20,55361,80.0,3.0,34.0,0.14,3.75,42.50
964,34_1,34,54518,2592,1919,490.0,4.75,74.04,18.90,54518,1817,1480,256.0,3.33,81.45,14.09,54518,452,382,79.0,0.83,84.51,17.48,54518,177.0,47.0,83.0,0.32,26.55,46.89,54518,46.0,8.0,29.0,0.08,17.39,63.04,54518,100.0,2.0,43.0,0.18,2.00,43.00
496,21_3,21,50018,3188,2229,590.0,6.37,69.92,18.51,50018,2421,1815,348.0,4.84,74.97,14.37,50018,489,356,109.0,0.98,72.80,22.29,50018,154.0,49.0,65.0,0.31,31.82,42.21,50018,38.0,5.0,24.0,0.08,13.16,63.16,50018,86.0,4.0,44.0,0.17,4.65,51.16
815,31_27,31,15445,589,491,74.0,3.81,83.36,12.56,15445,355,307,26.0,2.30,86.48,7.32,15445,148,134,13.0,0.96,90.54,8.78,15445,62.0,45.0,25.0,0.40,72.58,40.32,15445,11.0,4.0,5.0,0.07,36.36,45.45,15445,13.0,1.0,5.0,0.08,7.69,38.46


In [62]:
conversion_only = conversion_table_difficulty[['conv_to_start_full','conv_to_complete_full','conv_to_quit_full',\
    'conv_to_start_36','conv_to_complete_36','conv_to_quit_36',\
        'conv_to_start_49','conv_to_complete_49','conv_to_quit_49',\
           'conv_to_start_144','conv_to_complete_144','conv_to_quit_144',\
               'conv_to_start_256','conv_to_complete_256','conv_to_quit_256',\
                   'conv_to_start_441','conv_to_complete_441','conv_to_quit_441']]

In [63]:
conv_matrix_difficulty = pd.DataFrame(columns=['full', '36', '49', '144', '256', '441'], index = ['conv_to_start_difficulty', 'conv_to_complete_difficulty', 'conv_to_quit_difficulty'])
counter = 0
for i in range(6):
    x= []
    for j in range(3):
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        counter+=1
    conv_matrix_difficulty.iloc[:,i] = x
conv_matrix_difficulty

start_hits_list = ['starts_full', 'starts_36' ,'starts_49' ,'starts_144','starts_256', 'starts_441']
start_hits=[]
for i in start_hits_list:
    start_hits.append(round(conversion_table_difficulty[i].sum()*100/conversion_table_difficulty[start_hits_list[0]].sum(),1))
conv_matrix_difficulty.loc['starts hits, %'] = start_hits
conv_matrix_difficulty

,full,36,49,144,256,441
conv_to_start_difficulty,4.71,2.78,1.33,0.41,0.11,0.07
conv_to_complete_difficulty,85.34,89.22,90.52,61.89,39.83,10.29
conv_to_quit_difficulty,12.45,8.21,8.25,34.37,35.48,36.20
"starts hits, %",100.00,61.50,26.20,8.40,2.20,1.80


## Сохранимся в excel файл

In [64]:
with pd.ExcelWriter("16.05.22 Conversions_only_daily_catalog.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    result_matrix.to_excel(writer, sheet_name="Средние конверсий", index=False)
    conv_matrix_difficulty.to_excel(writer, sheet_name="Средние по сложностям", index=False)
    conversion_table_simply.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Полные конверсии", index=False)
    conversion_table_first.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Первый старт", index=False)
    conversion_table_first_plus.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Последующие старты", index=False)
    conversion_table_difficulty.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Сложности", index=False)

## Обратим внимание на странное значение Конверсии первого старта в копмлит с версией 1.4.3

In [76]:
first_starts_143 = data_work[data_work['app_version_name'] == '1.4.3'].query('event_name == "game_puzzle_complete"')
print('Количество первых комплитов ', first_starts_143.query('PuzzleComplete == 1').groupby(by='cat_puzzle_id').count().shape[0])

first_complets_143 = data_work[data_work['app_version_name'] == '1.4.3'].query('event_name == "game_puzzle_start"')
print('Количество первых стартов ',first_complets_143.query('PuzzleStart ==0').groupby(by='cat_puzzle_id').count().shape[0])

Количество первых комплитов  187
Количество первых стартов  251


Например такие же цифры для версии 1.4 будут выглядеть следующим образом

In [77]:
first_starts_14 = data_work[data_work['app_version_name'] == '1.4'].query('event_name == "game_puzzle_complete"')
print('Количество первых комплитов ', first_starts_14.query('PuzzleComplete == 1').groupby(by='cat_puzzle_id').count().shape[0])

first_complets_14 = data_work[data_work['app_version_name'] == '1.4'].query('event_name == "game_puzzle_start"')
print('Количество первых стартов ',first_complets_14.query('PuzzleStart ==0').groupby(by='cat_puzzle_id').count().shape[0])


Количество первых комплитов  1054
Количество первых стартов  1186
